In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/paolamateo/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


## Visit the NASA mars news site

In [3]:
# Visit the Mars news site
url = 'https://redplanetscience.com/'
browser.visit(url)

soup=bs(browser.html,'html.parser')

title=soup.find('div', class_="content_title").text
para=soup.find('div', class_="article_teaser_body").text

## JPL Space Images Featured Image

In [4]:
#Mars Featured Image
url= "https://spaceimages-mars.com"
browser.visit(url)
soup = bs(browser.html,'html.parser')

In [5]:
truncated_url=soup.find('img', class_="headerimage")["src"]
featured_img_url=f"{url}/{truncated_url}"

## Mars Facts

In [6]:
#Mars Facts
url= "https://galaxyfacts-mars.com"

In [7]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=df.iloc[0]
df=df[1:]
df.set_index(df.columns[0])
td= df.to_html()
td

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars - Earth Comparison</th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>1</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Temperature:</td>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

## Hemispheres

In [8]:
url = 'https://marshemispheres.com/'

browser.visit(url)
soup=bs(browser.html, 'html.parser')

In [9]:
h3=soup.find_all('h3')
four_h3= [tag.text for tag in h3][:-1]

In [10]:
# Create a list to hold the images and titles.
hemisphere_image_urls = []
browser.visit(url)

# Next, loop through those links, click the link, find the sample anchor, return the href
for hemi in range(len(four_h3)):
    hemi_dict = {}

    # We have to find the elements on each loop to avoid a stale element exception
    browser.click_link_by_partial_text(four_h3[hemi])
    
    soup= bs(browser.html, 'html.parser')

    trunc=soup.find('img', class_="wide-image")['src']
    hemi_dict["title"]=four_h3[hemi]
    hemi_dict["img_url"]=f"{url}/{trunc}"

    # Append hemisphere object to list
    hemisphere_image_urls.append(hemi_dict)
    
    # Finally, we navigate backwards
    browser.back()

/Applications/anaconda3/lib/python3.8/site-packages/splinter/driver/webdriver/__init__.py:519: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


In [11]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com//images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com//images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com//images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com//images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [12]:
browser.quit()